In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
from keras.models import Sequential
from keras import backend as K
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History 
history = History()
from sklearn import grid_search
from keras import backend as K
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')
import time
from datetime import datetime

seed = 42
np.random.seed(seed)

number_of_classes = 10

Using TensorFlow backend.
/Users/ram/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ram/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

5
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


In [3]:
def create_model (optimizer, dense_layer_sizes, filters, kernel_size, pool_size):
    
    now = datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    print ("optimizer : ", optimizer)
    print ("dense_layer_sizes : ", dense_layer_sizes)
    print ("filters : ", filters)
    print ("kernel_size : ", kernel_size)
    print ("pool_size : ", pool_size)
        
    model = Sequential()

    model.add(Conv2D(filters, kernel_size, input_shape=(28,28,1)))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))
  
    model.add(Conv2D(filters*2, kernel_size))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(Conv2D(filters*2, kernel_size))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    BatchNormalization()
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.25))
    model.add(Dense(10))
    
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.summary()
    return model

In [4]:
model_clf = KerasClassifier(create_model, batch_size=32)

# Grid search 
optimizers = ['rmsprop', 'adam']
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [50, 100, 150]
#activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
dense_layer_sizes =  [32, 512]
filters = [32]
kernel_size = [3]
pool_size = [2]

param_grid = dict(optimizer=optimizers, dense_layer_sizes=dense_layer_sizes,filters=filters,kernel_size=kernel_size,pool_size=pool_size)

grid_clf  = GridSearchCV(model_clf,
                         param_grid=param_grid,
                         scoring='neg_log_loss',
                         n_jobs=1)
grid_clf.fit(X_train, Y_train)
print('The parameters of the best model are: ')
print(grid_clf.best_params_)

'''
# Evaluation using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model_clf, X_train, Y_train, cv=kfold)
print(results.mean())
'''

Current date and time : 
2017-11-22 23:13:15
('optimizer : ', 'rmsprop')
('dense_layer_sizes : ', 32)
('filters : ', 32)
('kernel_size : ', 3)
('pool_size : ', 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 3

Epoch 1/10
40000/40000 [==============================] - 253s - loss: 0.7704 - acc: 0.7480   
Epoch 2/10
40000/40000 [==============================] - 221s - loss: 0.2948 - acc: 0.9260   
Epoch 3/10
40000/40000 [==============================] - 229s - loss: 0.2125 - acc: 0.9504   
Epoch 4/10
40000/40000 [==============================] - 248s - loss: 0.1852 - acc: 0.9581   
Epoch 5/10
40000/40000 [==============================] - 236s - loss: 0.1709 - acc: 0.9634   
Epoch 6/10
40000/40000 [==============================] - 225s - loss: 0.1711 - acc: 0.9650   
Epoch 7/10
40000/40000 [==============================] - 227s - loss: 0.1700 - acc: 0.9645   
Epoch 8/10
40000/40000 [==============================] - 216s - loss: 0.1724 - acc: 0.9657   
Epoch 9/10
40000/40000 [==============================] - 215s - loss: 0.1737 - acc: 0.9673   
Epoch 10/10
20000/20000 [==============================] - 32s    
Current date and time : 
2017-11-23 00:16:40
('optimizer : ', 'rmsprop')
('den

Epoch 1/10
40000/40000 [==============================] - 130s - loss: 0.7072 - acc: 0.7595   
Epoch 2/10
40000/40000 [==============================] - 130s - loss: 0.2368 - acc: 0.9385   
Epoch 3/10
40000/40000 [==============================] - 130s - loss: 0.1846 - acc: 0.9536   
Epoch 4/10
40000/40000 [==============================] - 129s - loss: 0.1522 - acc: 0.9628   
Epoch 5/10
40000/40000 [==============================] - 129s - loss: 0.1229 - acc: 0.9684   
Epoch 6/10
40000/40000 [==============================] - 130s - loss: 0.1188 - acc: 0.9713   - ETA: 0s - loss: 0.1189 - acc: 0
Epoch 7/10
40000/40000 [==============================] - 130s - loss: 0.1095 - acc: 0.9732   
Epoch 8/10
40000/40000 [==============================] - 130s - loss: 0.0999 - acc: 0.9748   
Epoch 9/10
40000/40000 [==============================] - 130s - loss: 0.0899 - acc: 0.9781   
Epoch 10/10
20000/20000 [==============================] - 18s    
Current date and time : 
2017-11-23 01:05:27


Epoch 1/10
40000/40000 [==============================] - 131s - loss: 0.7829 - acc: 0.7336   
Epoch 2/10
40000/40000 [==============================] - 130s - loss: 0.2827 - acc: 0.9242   
Epoch 3/10
40000/40000 [==============================] - 130s - loss: 0.2078 - acc: 0.9458   
Epoch 4/10
40000/40000 [==============================] - 130s - loss: 0.1587 - acc: 0.9603   
Epoch 5/10
40000/40000 [==============================] - 130s - loss: 0.1409 - acc: 0.9632   
Epoch 6/10
40000/40000 [==============================] - 129s - loss: 0.1253 - acc: 0.9683   
Epoch 7/10
40000/40000 [==============================] - 130s - loss: 0.1173 - acc: 0.9700   
Epoch 8/10
40000/40000 [==============================] - 130s - loss: 0.1097 - acc: 0.9726   
Epoch 9/10
40000/40000 [==============================] - 129s - loss: 0.1054 - acc: 0.9740   
Epoch 10/10
20000/20000 [==============================] - 18s    
Current date and time : 
2017-11-23 01:50:37
('optimizer : ', 'rmsprop')
('den

Epoch 1/10
40000/40000 [==============================] - 151s - loss: 0.2952 - acc: 0.9135   
Epoch 2/10
40000/40000 [==============================] - 151s - loss: 0.1486 - acc: 0.9673   
Epoch 3/10
40000/40000 [==============================] - 151s - loss: 0.1562 - acc: 0.9696   
Epoch 4/10
40000/40000 [==============================] - 151s - loss: 0.1716 - acc: 0.9682   
Epoch 5/10
40000/40000 [==============================] - 151s - loss: 0.1865 - acc: 0.9678   
Epoch 6/10
40000/40000 [==============================] - 151s - loss: 0.2120 - acc: 0.9650   
Epoch 7/10
40000/40000 [==============================] - 151s - loss: 0.2398 - acc: 0.9640   
Epoch 8/10
40000/40000 [==============================] - 151s - loss: 0.2519 - acc: 0.9631   
Epoch 9/10
40000/40000 [==============================] - 151s - loss: 0.3106 - acc: 0.9596   
Epoch 10/10
20000/20000 [==============================] - 21s    
Current date and time : 
2017-11-23 02:41:57
('optimizer : ', 'rmsprop')
('den

Epoch 1/10
40000/40000 [==============================] - 157s - loss: 0.2404 - acc: 0.9254   
Epoch 2/10
40000/40000 [==============================] - 156s - loss: 0.0906 - acc: 0.9768   
Epoch 3/10
40000/40000 [==============================] - 156s - loss: 0.0770 - acc: 0.9805   
Epoch 4/10
40000/40000 [==============================] - 155s - loss: 0.0664 - acc: 0.9832   
Epoch 5/10
40000/40000 [==============================] - 155s - loss: 0.0637 - acc: 0.9845   
Epoch 6/10
40000/40000 [==============================] - 156s - loss: 0.0590 - acc: 0.9857   
Epoch 7/10
40000/40000 [==============================] - 157s - loss: 0.0476 - acc: 0.9879   
Epoch 8/10
40000/40000 [==============================] - 156s - loss: 0.0454 - acc: 0.9897   
Epoch 9/10
40000/40000 [==============================] - 156s - loss: 0.0464 - acc: 0.9888   
Epoch 10/10
20000/20000 [==============================] - 22s    
Current date and time : 
2017-11-23 03:34:15
('optimizer : ', 'adam')
('dense_

Epoch 1/10
40000/40000 [==============================] - 157s - loss: 0.2448 - acc: 0.9259   
Epoch 2/10
40000/40000 [==============================] - 156s - loss: 0.0934 - acc: 0.9758   
Epoch 3/10
40000/40000 [==============================] - 156s - loss: 0.0784 - acc: 0.9805   
Epoch 4/10
40000/40000 [==============================] - 156s - loss: 0.0655 - acc: 0.9838   
Epoch 5/10
40000/40000 [==============================] - 156s - loss: 0.0621 - acc: 0.9849   
Epoch 6/10
40000/40000 [==============================] - 156s - loss: 0.0578 - acc: 0.9861   
Epoch 7/10
40000/40000 [==============================] - 156s - loss: 0.0490 - acc: 0.9876   
Epoch 8/10
40000/40000 [==============================] - 156s - loss: 0.0438 - acc: 0.9892   
Epoch 9/10
40000/40000 [==============================] - 156s - loss: 0.0524 - acc: 0.9873   
Epoch 10/10
20000/20000 [==============================] - 22s    
Current date and time : 
2017-11-23 04:27:19
('optimizer : ', 'adam')
('dense_

The parameters of the best model are: 
{'dense_layer_sizes': 32, 'optimizer': 'adam', 'kernel_size': 3, 'filters': 32, 'pool_size': 2}

In [ ]:
# summarize results
best_clf = grid_clf.best_estimator_.model

metric_names = best_clf.metrics_names
metric_values = best_clf.evaluate(X_test, Y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)    